<a href="https://colab.research.google.com/github/mocci24/RVCGI-V2/blob/main/MrMocci_RVCV2_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HOW?**

### In the process of creating an AI cover, there are several stages involved :

1. Separating the vocals and music from the song.
2. Performing inference (transforming it into AI-generated vocals) on the separated vocals.
3. Rejoining the AI-generated vocals with the original music (for this last step, I recommend using third-party apps like FL STUDIO, BandLab, and others).

the end, Adios ✨

# PREPARATION

In [ ]:
!nvidia-smi
!nvcc -V
!free -h

# DOWNLOAD THE SONG FROM YOUTUBE



In [ ]:
#@title Installation
!pip install yt_dlp
!pip install ffmpeg
!mkdir youtubeaudio

In [ ]:
#@title Download Youtube WAV
from __future__ import unicode_literals
import yt_dlp
import ffmpeg
import sys

#@markdown Don't give space for song name
url = "" #@param {type:"string"}
song_name = "" #@param {type:"string"}

ydl_opts = {
    'format': 'bestaudio/best',
#    'outtmpl': 'output.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    "outtmpl": f'youtubeaudio/{song_name}',  # this is where you can edit how you'd like the filenames to be formatted
}
def download_from_url(url):
    ydl.download([url])
    # stream = ffmpeg.input('output.m4a')
    # stream = ffmpeg.output(stream, 'output.wav')


with yt_dlp.YoutubeDL(ydl_opts) as ydl:


      download_from_url(url)

# SEPARATE VOCAL & MUSIC

In [ ]:
#@title Install Demucs for Separating Audio
!python3 -m pip install -U demucs

In [ ]:
#@title Start Separate Vocal and Instrument using Demucs
#@markdown This program is standart separating Vocal and Instrument.
#@markdown Use webUI for more feature.
import subprocess
input_file = "" #@param {type:"string"}
OUTPUT_DIR = "/content/separated"

command = f"demucs --two-stems=vocals {input_file}"
result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())
#input_file = input("Masukkan file audio: ")

# AUDIO SPLIT PER-ONE MINUTE SEGMENT

In [ ]:
#@title Instal requirements for audio splitter ( 1 minute per split )
!pip install pydub

In [ ]:
#@title Start Splitter ( 1-minute segments )
#@markdown Input file path, for example
#@markdown "/content/separated/vocal.wav"
#@markdown
#@markdown - What is this for?<br />
#@markdown To split the vocal duration into one-minute segments.<br />
#@markdown - Why does it need to be split?<br />
#@markdown To avoid "connection timeout" messages during inference when using the "Havest" mode, as the maximum duration that can be processed is only around one minute.<br />
#@markdown However, if you are using the PM mode, you can skip this process.
input_file = "" #@param {type:"string"}
from pydub import AudioSegment
import os

def split_audio_per_minute(input_file):
    output_directory = '/content/Split'
    os.makedirs(output_directory, exist_ok=True)
    audio = AudioSegment.from_file(input_file)
    duration = len(audio)
    total_minutes = int(duration / (60 * 1000))

    for minute in range(total_minutes + 1):
        start_time = minute * 60 * 1000
        end_time = (minute + 1) * 60 * 1000
        split_audio = audio[start_time:end_time]
        output_file = os.path.join(output_directory, f'split_{os.path.splitext(os.path.basename(input_file))[0]}_{minute}.wav')
        split_audio.export(output_file, format='wav')
        print(f"Berhasil memisahkan audio {os.path.basename(input_file)} menit ke-{minute + 1}.")

    print("Proses pemisahan audio selesai.")

#input_file = input("Masukkan file audio: ")
split_audio_per_minute(input_file)

# LOAD MODELS

In [ ]:
#@title Load models from Hugingface
#@markdown Click [here](https://huggingface.co/mocci24/RVCV2-GI) for more info
!apt install git-lfs
!git lfs install
!git clone https://huggingface.co/mocci24/RVCV2-GI

In [ ]:
#@title Cloning Github | Hugingface
#@markdown Check our Models info
#@markdown [Here](https://huggingface.co/mocci24/RVCV2-GI)


!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs
!cp -r /content/RVCV2-GI/req/* /content/Retrieval-based-Voice-Conversion-WebUI
!mv /content/RVCV2-GI/RVC/* /content/Retrieval-based-Voice-Conversion-WebUI/logs
!mv /content/Retrieval-based-Voice-Conversion-WebUI/logs/weights/* /content/Retrieval-based-Voice-Conversion-WebUI/weights
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

# INSTALL REQUIREMENTS FOR WebUI

In [ ]:
!pip install gradio==3.36.1
!pip install -r requirements-full.txt

In [ ]:
#@title Install aria2
!apt -y install -qq aria2

In [ ]:
#@title Download requirements for Audio Splitting
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/5_HP-Karaoke-UVR.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o 5_HP-Karaoke-UVR.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP3_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP3_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5_only_main_vocal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5_only_main_vocal.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoAggressive.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoAggressive.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoDeReverb.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoDeReverb.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoNormal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoNormal.pth

In [ ]:
#@title Download hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o rmvpe.pt

# RUN WebUI

In [ ]:
#@title Run WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3

# <center> Confused? Yes Me Too<br />
# <center> Some [☕](https://ko-fi.com/mocci24)?